In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mc3.stats import time_avg
from glob import glob
import yaml
import warnings
warnings.filterwarnings("ignore")

plt.rcParams.update({
    'font.size': 18,
    'font.family': 'serif',
    'axes.linewidth': 1.2,
    'xtick.major.size': 5,
    'ytick.major.size': 5,
    'xtick.minor.size': 3,
    'ytick.minor.size': 3,
    'axes.labelpad': 8,
    'lines.linewidth': 1.5,
    'lines.markersize': 8,
    'axes.titlepad': 12,
    'legend.frameon': True,
    'legend.framealpha': 0.8
})

In [2]:
def plot_time_avg(files, labels, outdir=''):
    # Initialize the figure
    fig = plt.figure(figsize=(10, 7))
    fig.clf()
    ax = fig.gca()

    # Color setup
    cmap = plt.cm.get_cmap('tab10')
    color_dict = {label: cmap(i/len(labels)) for i, label in enumerate(labels)}

    # Plot each file
    for f, l in zip(files, labels):
        results = pd.read_csv(f, comment='#', sep=' ')
        residuals = np.array(results['residuals'])
        time = np.array(results['time'])
        
        # Mask invalid values and sort
        time = np.ma.masked_invalid(time)
        time = time[~np.ma.getmaskarray(time)]
        residuals = np.ma.masked_invalid(residuals)
        residuals = residuals[np.ma.argsort(time)]
        residuals = residuals[~np.ma.getmaskarray(residuals)]
        
        # Bin calculations
        maxbins = residuals.size // 10
        maxbins = max(maxbins, 2)  # Ensure at least 2 bins
        rms, rmslo, rmshi, stderr, binsz = time_avg(residuals, maxbins=maxbins, binstep=1)
        
        normfactor = 1e-6  # Normalization to ppm
        
        # Plot RMS and uncertainty
        ax.plot(binsz, rms/normfactor, color=color_dict[l], lw=2, label=f'{l} Fit RMS', zorder=4)
        ax.fill_between(binsz, (rms-rmslo)/normfactor, (rms+rmshi)/normfactor,
                        color=color_dict[l], alpha=0.2, zorder=3)
        
        # Plot Gaussian std error
        ax.plot(binsz, stderr/normfactor, color=color_dict[l], ls='--', lw=2,
                # label=f'{l} Gaussian Std. Err.', zorder=2)
                label=f'{l} White Noise', zorder=2)

    # Axes settings
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel("Bin Size [N frames]", fontsize=14)
    ax.set_ylabel("Residual RMS [ppm]", fontsize=14)
    # ax.tick_params(axis='both', labelsize=12)
    ax.tick_params(direction='in', which='both', labelsize=12, right=True)

#     ax.set_title('Correlated Noise Analysis', fontsize=16, pad=20)

    # Add grid
    ax.grid(True, which='both', ls=':', alpha=0.6, zorder=0)

    # Legend customization
    # handles, labels = ax.get_legend_handles_labels()
    # Remove duplicate labels (arising from multiple entries per label)
    # unique_labels = dict(zip(labels, handles))
    # ax.legend(unique_labels.values(), unique_labels.keys(),
            #   loc='upper center', bbox_to_anchor=(0.5, -0.12),
    ax.legend(
            loc='upper right',
            fontsize=10, ncol=2, frameon=False, fancybox=True, shadow=False)

    # Secondary x-axis for time in seconds
    dt = np.ma.min(np.ma.diff(time)) * 24 * 3600  # Convert days to seconds
    ax2 = ax.secondary_xaxis('top', functions=(lambda N: N*dt, lambda t: t/dt))
    ax2.set_xlabel('Bin Size [seconds]', fontsize=14)
#     ax2.tick_params(axis='both', labelsize=12)
    ax2.tick_params(direction='in', which='both', labelsize=12)


    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'{outdir}residual_rms.png', dpi=300)
    plt.savefig(f'{outdir}residual_rms.pdf', dpi=600)
    # plt.show()
    plt.close(fig)

In [3]:
def plot_time_avg_relative(files, labels, outdir=''):
        # Initialize the figure
        fig = plt.figure(figsize=(10, 7))
        fig.clf()
        ax = fig.gca()

        # Color setup
        cmap = plt.cm.get_cmap('tab10')
        color_dict = {label: cmap(i/len(labels)) for i, label in enumerate(labels)}

        # Plot each file
        for f, l in zip(files, labels):
                results = pd.read_csv(f, comment='#', sep=' ')
                residuals = np.array(results['residuals'])
                time = np.array(results['time'])
                
                # Mask invalid values and sort
                time = np.ma.masked_invalid(time)
                time = time[~np.ma.getmaskarray(time)]
                residuals = np.ma.masked_invalid(residuals)
                residuals = residuals[np.ma.argsort(time)]
                residuals = residuals[~np.ma.getmaskarray(residuals)]

                # Calculate unbinned RMS (reference level)
                sigma = np.sqrt(np.mean(residuals**2))  # RMS of unbinned residuals
                # Bin calculations
                maxbins = residuals.size // 10
                maxbins = max(maxbins, 2)  # Ensure at least 2 bins
                rms, rmslo, rmshi, stderr, binsz = time_avg(residuals, maxbins=maxbins, binstep=1)
                
                # normfactor = 1e-6  # Normalization to ppm
                
                # Calculate relative RMS and uncertainties
                relative_rms = rms / sigma
                relative_rmslo = (rms - rmslo) / sigma
                relative_rmshi = (rms + rmshi) / sigma
                
                # Plot RMS and uncertainty
                ax.plot(binsz, relative_rms, color=color_dict[l], lw=2, label=f'{l} Fit RMS', zorder=4)
                ax.fill_between(binsz, relative_rmslo, relative_rmshi,
                                color=color_dict[l], alpha=0.2, zorder=3)
                        
                # # Plot Gaussian std error
                # ax.plot(binsz, stderr/normfactor, color=color_dict[l], ls='--', lw=2,
                #         # label=f'{l} Gaussian Std. Err.', zorder=2)
                #         label=f'{l} White Noise', zorder=2)

        # Plot theoretical white noise curve
        xlim = ax.get_xlim()
        # n = np.logspace(np.log10(xlim[0]), np.log10(xlim[1]), 100)
        ax.plot(range(int(xlim[-1])), 1/np.sqrt(range(int(xlim[-1]))), color='gray', ls='--', lw=2, 
                # label='White Noise (1/√N)', zorder=2)
                label='White Noise', zorder=2)

        # Axes settings
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel("Bin Size [N frames]", fontsize=14)
        ax.set_ylabel("Relative Residual RMS", fontsize=14)
        # ax.tick_params(axis='both', labelsize=12)
        ax.tick_params(direction='in', which='both', labelsize=12, right=True)

        # ax.set_title('Correlated Noise Analysis', fontsize=16, pad=20)

        # Add grid
        ax.grid(True, which='both', ls=':', alpha=0.6, zorder=0)

        # Legend customization
        # handles, labels = ax.get_legend_handles_labels()
        # Remove duplicate labels (arising from multiple entries per label)
        # unique_labels = dict(zip(labels, handles))
        # ax.legend(unique_labels.values(), unique_labels.keys(),
        ax.legend(
                #   loc='upper center', bbox_to_anchor=(0.5, -0.12),
                loc='lower left',
                fontsize=10, ncol=1, frameon=False, fancybox=True, shadow=False)

        # Secondary x-axis for time in seconds
        dt = np.ma.min(np.ma.diff(time)) * 24 * 3600  # Convert days to seconds
        ax2 = ax.secondary_xaxis('top', functions=(lambda N: N*dt, lambda t: t/dt))
        ax2.set_xlabel('Bin Size [seconds]', fontsize=14)
        # ax2.tick_params(axis='both', labelsize=12)
        ax2.tick_params(direction='in', which='both', labelsize=12)

        # Adjust layout
        plt.tight_layout()
        plt.savefig(f'{outdir}residual_rms_relative.png', dpi=300)
        plt.savefig(f'{outdir}residual_rms_relative.pdf', dpi=600)
        plt.close(fig)

In [4]:
# name = 'tess'
name = 'jwst'
with open(f'init_{name}.yaml', 'r') as f:
    init = yaml.safe_load(f)

In [5]:
for i,p in enumerate(init):
    print(f'Processing {i+1}/{len(init)}: {p}')
    IDs = init[p]['IDs']
    instrument = init[p]['instrument']
    labels = [f'{id}-{ins}' for id, ins in zip(IDs, instrument)]
    indir = f'wasp-107b/{name}/'
    files = sorted(glob(f'*/{name}/*/S5_*Table_Save*.txt'))
    plot_time_avg(files, labels, indir)
    plot_time_avg_relative(files, labels, indir)

Processing 1/1: WASP-107b
